In [24]:
import pandas as pd 
import numpy as np 
import os 
import pathlib 
from tqdm import tqdm

maindir = "/Users/hieunguyen/data/WGS_features/20240929"
path_to_feature = os.path.join(maindir, "feature")
path_to_metadata = os.path.join(maindir, "metadata", "metadata.xlsx")

##### read feature end motif to check
if os.path.isfile("em.old.csv") == False:
    feat = "EM"
    maindf = pd.DataFrame(
        data = [f"{i}{j}{k}{l}"
                for i in ["A", "T", "G", "C"]
                for j in ["A", "T", "G", "C"]
                for k in ["A", "T", "G", "C"]
                for l in ["A", "T", "G", "C"]],
        columns = ["motif"]
    )
    all_files = [item for item in pathlib.Path(os.path.join(path_to_feature)).glob(f"*/*GWfeature_{feat}.csv")]
    for file in tqdm(all_files):
        sampleid = file.name.split("_")[0]
        tmpdf = pd.read_csv(file, index_col = [0])
        tmpdf.columns = ["motif", sampleid]
        maindf = maindf.merge(tmpdf, right_on = "motif", left_on = "motif")
        
    emdf = maindf[["motif", "1-ZLAAO90NB"]]
    emdf.columns =  ["size", "ZLAAO90NB"]
    emdf.to_csv("em.old.csv")
else:
    emdf = pd.read_csv("em.old.csv", index_col = [0])

##### read feature nucleosome to check
if os.path.isfile("nuc.old.csv") == False:
    feat = "Nucleosome"
    maindf = pd.DataFrame(
        data = range(-300, 301),
        columns = ["size"]
    )
    maindf["size"] = maindf["size"].astype(int)
    all_files = [item for item in pathlib.Path(os.path.join(path_to_feature)).glob(f"*/*GWfeature_{feat}.csv")]
    for file in tqdm(all_files):
        sampleid = file.name.split("_")[0]
        tmpdf = pd.read_csv(file, index_col = [0])
        tmpdf = tmpdf.T.reset_index()
        tmpdf.columns = ["size", sampleid]
        tmpdf["size"] = tmpdf["size"].astype(int)
        maindf = maindf.merge(tmpdf, right_on = "size", left_on = "size")
        
    nucdf = maindf[["size", "1-ZLAAO90NB"]]
    nucdf.columns =  ["size", "ZLAAO90NB"]
    nucdf.to_csv("nuc.old.csv")
else:
    nucdf = pd.read_csv("nuc.old.csv", index_col = [0])
    
feat = "flen"
if os.path.isfile("flen.old.csv") == False:
    maindf = pd.DataFrame(data = range(50, 351), columns = ["size"])
    maindf["size"] = maindf["size"].astype(int)
    all_files = [item for item in pathlib.Path(os.path.join(path_to_feature)).glob(f"*/*GWfeature_{feat}.csv")]
    for file in tqdm(all_files):
            sampleid = file.name.split("_")[0]
            tmpdf = pd.read_csv(file, index_col = [0])[["size", "freq"]]
            tmpdf.columns = ["size", sampleid]
            maindf = maindf.merge(tmpdf, right_on = "size", left_on = "size")
    flendf = maindf[["size", "1-ZLAAO90NB"]]
    flendf.columns = ["size", "ZLAAO90NB"]
    flendf.to_csv("flen.old.csv")
else:
    flendf = pd.read_csv("flen.old.csv", index_col = [0])

100%|██████████| 1878/1878 [00:11<00:00, 167.33it/s]


KeyError: "['ZLAAO90NB'] not in index"